In [2]:
import os 
from importlib import reload  
import numpy as np
import torch,time
from transformers import BartModel,BartConfig,BartForConditionalGeneration,BartForCausalLM, BartTokenizer
from tqdm.notebook import tqdm
from torch import nn

In [3]:
import sys

MOD_FOLDER = '/work/05773/anubrata/ls6/ProtoTEx/'
# setting path to enable import from the parent directory
sys.path.append(MOD_FOLDER)
print(sys.path)

['/work/05773/anubrata/ls6/ProtoTEx/Notebooks', '/work/05773/anubrata/ls6/anaconda3/envs/prototex/lib/python38.zip', '/work/05773/anubrata/ls6/anaconda3/envs/prototex/lib/python3.8', '/work/05773/anubrata/ls6/anaconda3/envs/prototex/lib/python3.8/lib-dynload', '', '/work/05773/anubrata/ls6/anaconda3/envs/prototex/lib/python3.8/site-packages', '/work/05773/anubrata/ls6/ProtoTEx/']


In [4]:
from models import SimpleProtoTex
from models import ProtoTEx

In [5]:
num_prototypes = 20
num_pos_prototypes = 19
model = ProtoTEx(num_prototypes, 
                 num_pos_prototypes,
                 bias=False, 
                 dropout=False, 
                 special_classfn=True, # special_classfn=False, ## apply dropouonly on bias 
                 p=1, #p=0.75,
                 batchnormlp1=True)
# model

In [20]:
model_path = "Models/NegProtoTEx_protos_xavier_large_bs20_20_woRat_noReco_g2d_nobias_nodrop_cu1_PosUp_normed"

In [21]:
model.load_state_dict(torch.load(MOD_FOLDER + model_path))

<All keys matched successfully>

In [13]:
device = torch.device('cuda:0')
model.to(device)

ProtoTEx(
  (bart_model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50265, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0): BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1

In [7]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

In [8]:
## Load all the functions for analyzing prototypes
from utils import *

In [9]:
## Load all the data classes
from preprocess import *

In [10]:
## Load data
train=make_dataset(pathlib.Path("../data/protechn_corpus_eval/train/"))
val=make_dataset(pathlib.Path("../data/protechn_corpus_eval/dev/"))
test=make_dataset(pathlib.Path("../data/protechn_corpus_eval/test/"))

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

train_=make_bert_testset(train)
val_=make_bert_testset(val)
test_=make_bert_testset(test)
# train_sents=[ " ".join(i) for d in train_[0] for i in d]
train_sents=[ list(map(lambda x: x[1] if x[0]==0 else " "+x[1], enumerate(i))) for d in train_[0] for i in d]
# val_sents=[ " ".join(i) for d in val_[0] for i in d]
val_sents=[ list(map(lambda x: x[1] if x[0]==0 else " "+x[1], enumerate(i))) for d in val_[0] for i in d]
# test_sents=[ " ".join(i) for d in test_[0] for i in d]
test_sents=[ list(map(lambda x: x[1] if x[0]==0 else " "+x[1], enumerate(i))) for d in test_[0] for i in d]


train_ls=create_labels(train_)
val_ls=create_labels(val_)
test_ls=create_labels(test_)
train_y_txt=[ i for d in train_[1] for i in d]
val_y_txt=[ i for d in val_[1] for i in d]
test_y_txt=[ i for d in test_[1] for i in d]


train_idx_bylabel={x: [i for i in range(len(train_ls)) if train_ls[i]==x] for x in labels_set} 
val_idx_bylabel={x: [i for i in range(len(val_ls)) if val_ls[i]==x] for x in labels_set} 
test_idx_bylabel={x: [i for i in range(len(test_ls)) if test_ls[i]==x] for x in labels_set} 

train_dataset=BinaryClassDataset(train_sents,train_ls,train_y_txt,it_is_train=0,balance=True, tokenizer=tokenizer)
val_dataset=BinaryClassDataset(val_sents,val_ls,val_y_txt,it_is_train=0, tokenizer=tokenizer)
test_dataset=BinaryClassDataset(test_sents,test_ls,test_y_txt,it_is_train=0, tokenizer=tokenizer)
train_dataset_eval=BinaryClassDataset(train_sents,train_ls,train_y_txt,it_is_train=0,balance=False, tokenizer=tokenizer)


train_dl=torch.utils.data.DataLoader(train_dataset,batch_size=20,shuffle=True,
                                 collate_fn=train_dataset.collate_fn)
val_dl=torch.utils.data.DataLoader(val_dataset,batch_size=128,shuffle=False,
                                 collate_fn=val_dataset.collate_fn)
test_dl=torch.utils.data.DataLoader(test_dataset,batch_size=128,shuffle=False,
                                 collate_fn=test_dataset.collate_fn)
train_dl_eval=torch.utils.data.DataLoader(train_dataset_eval,batch_size=20,shuffle=False,
                                 collate_fn=train_dataset_eval.collate_fn)

In [15]:
best_protos_per_testeg = get_best_k_protos_for_batch(test_sents, 
                                                   test_ls, 
                                                   test_y_txt, 
                                                   specific_label=None, 
                                                   model_new=model, 
                                                   tokenizer=tokenizer, topk= 5, do_all=True)

100%|██████████| 32/32 [00:06<00:00,  5.13batches/s]


In [16]:
train_sents_joined=[" ".join(i) for i in train_sents]
test_sents_joined=[" ".join(i) for i in test_sents]

In [18]:
"""
distances generation
test true labels and pred labels 
"""
loader = tqdm(test_dl, total=len(test_dl), unit="batches")
model.eval()    
with torch.no_grad():
    test_true=[]
    test_pred=[]
    for batch in loader:
        input_ids,attn_mask,y=batch
        classfn_out,_=model(input_ids,attn_mask,y,use_decoder=False,use_classfn=1)
        predict=torch.argmax(classfn_out,dim=1)
#         correct_idxs.append(torch.nonzero((predicted==y.cuda())).view(-1)
        test_pred.append(predict.cpu().numpy())
        test_true.append(y.cpu().numpy())
test_true=np.concatenate(test_true)
test_pred=np.concatenate(test_pred)


  0%|          | 0/32 [00:22<?, ?batches/s]

100%|██████████| 32/32 [00:06<00:00,  4.91batches/s]


In [40]:
bestk_train_data_per_proto=get_bestk_train_data_for_every_proto(train_dataset_eval, 
                                                   model_new=model, top_k=5)


"""
distances generation
csv generation
"""
import csv

fields = ["S.No.", "Test Sentence","Predicted","Actual","Actual Prop or NonProp"]
num_protos_per_test=5
num_train_per_proto=5
for i in range(num_protos_per_test):
    for j in range(num_train_per_proto):
        fields.append(f"Prototype_{i}_wieght0")
        fields.append(f"Prototype_{i}_wieght1")
        fields.append(f"Prototype_{i}_Nearest_train_eg_{j}")
        fields.append(f"Prototype_{i}_Nearest_train_eg_{j}_actuallabel")
        fields.append(f"Prototype_{i}_Nearest_train_eg_{j}_distance")
        
filename = f"{model_path[len('Models/'):]}_nearest.csv"
weights=model.classfn_model.weight.detach().cpu().numpy()
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for i in range(len(test_sents_joined)):
#     for i in range(100):
        row=[i,test_sents_joined[i],test_pred[i],test_ls[i],test_true[i]]
        for j in range(num_protos_per_test):
            proto_idx=best_protos_per_testeg[0][i][j]
            for k in range(num_train_per_proto):
#                 print(i,j,k)
                row.append(weights[0][proto_idx])
                row.append(weights[1][proto_idx])
                row.append(train_sents_joined[bestk_train_data_per_proto[0][proto_idx][k]])
                row.append(train_ls[bestk_train_data_per_proto[0][proto_idx][k]])
                row.append(bestk_train_data_per_proto[1][k][proto_idx])

        csvwriter.writerow(row)

100%|██████████| 112/112 [00:22<00:00,  5.06batches/s]
